<a href="https://colab.research.google.com/github/JTT94/nlp_neural_process/blob/master/Class_BERT_NeuralProcesses_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

W0528 09:22:55.800379 139661544327040 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 2.8MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab


# Get TensorBoard running in the background. 
LOG_DIR = './gdrive/My Drive/Kaggle_toxic_comments/test_output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
# # #Download and unzip ngrok. 
!test -e ngrok-stable-linux-amd64.zip || wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!test -e ngrok || unzip ngrok-stable-linux-amd64.zip


--2019-05-28 09:23:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.4.95.48, 34.226.180.131, 52.55.191.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.4.95.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  27.7MB/s    in 0.6s    

2019-05-28 09:23:31 (27.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d5a9c0ab.ngrok.io


In [0]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_model_builder_AUC import *
from neural_process.bert_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 129 (delta 77), reused 102 (delta 61), pack-reused 0
Receiving objects: 100% (129/129), 721.42 KiB | 2.05 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [0]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
# filename = './cleaned_data.csv'
# filename = './data1.csv'
# df = pd.read_csv(filename, index_col=0)
# cols = ['comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# cols = ['comment','cleaned_comment','antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# df = df[cols]

#score_column = ['antagonize', 'condescending', 'dismissive', 'generalisation', 'hostile', 'sarcastic', 'healthy']
# text_col_name = 'cleaned_comment'
# text_col_name = 'comment'

#--------

### For kaggle dataset

## Training data

# filename = './gdrive/My Drive/Data/train.csv'
train_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_train.csv'

df = pd.read_csv(train_filename)
cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[cols]

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

text_col_name = 'comment_text'

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df[i] = pd.to_numeric(df[i],downcast='float')

# over represent toxic comments

df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

df_toxic = df[df.num_toxic_atts  > 0]
# print(len(df_toxic))

df_healthy_sample = df[df.num_toxic_atts == 0].sample(frac=1.0)[:len(df_toxic)]
# print(len(df_healthy_sample))

df_train = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)


## Test data:

test_data_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test.csv'
test_labels_filename = './gdrive/My Drive/Kaggle_toxic_comments/kaggle_test_labels.csv'
test_df = pd.read_csv(test_data_filename)
test_labels = pd.read_csv(test_labels_filename)
test_df = pd.merge(test_df, test_labels, on='id')
#get rid of -1 values
test_df = test_df.replace(-1, np.nan).dropna(subset=cols)

#restrict comment length
test_df = test_df[test_df.comment_text.str.len() <= 250]

test_df = test_df[cols]
#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  test_df[i] = pd.to_numeric(test_df[i],downcast='float')

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

Instructions for updating:
Colocations handled automatically by placer.


W0528 09:24:07.873405 139661544327040 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 09:24:09.987522 139661544327040 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# #Pre process data for bert embedding
max_seq_length = 128

import pickle

# train_input_examples = create_examples(df_train.sample(100), score_column, text_col_name)
# # # test_input_examples = create_examples(df_test, score_column, text_col_name)

# train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
# # # test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

# pickle.dump(train_features, open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'wb'))
# # pickle.dump(test_features, open('./gdrive/My Drive/Kaggle_toxic_comments/test_features.p', 'wb'))

# # ## Load features previously saved
# train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
# test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:
# utility methods
from collections import namedtuple
NeuralProcessParams = namedtuple('NeuralProcessParams', ['dim_z', 'n_hidden_units_h', 'n_hidden_units_g'])
GaussianParams = namedtuple('GaussianParams', ['mu', 'sigma'])


def batch_mlp(input, inner_layer_dims, output_dim, variable_scope):
  """Apply MLP to the final axis of a 3D tensor (reusing already defined MLPs).
  
  Args:
    input: input tensor of shape [B,n,d_in].
    output_sizes: An iterable containing the output sizes of the MLP as defined 
        in `basic.Linear`.
    variable_scope: String giving the name of the variable scope. If this is set
        to be the same as a previously defined MLP, then the weights are reused.
    
  Returns:
    tensor of shape [B,n,d_out] where d_out=output_sizes[-1]
  """
  # Get the shapes of the input and reshape to parallelise across observations

  output = input

  
  # Pass through MLP
  with tf.variable_scope(variable_scope, reuse=tf.AUTO_REUSE):
    for i, size in enumerate(inner_layer_dims):
      output = tf.nn.relu(
          tf.layers.dense(output, size, name="layer_{}".format(i)))

    # Last layer without a ReLu
    output = tf.layers.dense(output, output_dim, name="layer_{}".format(i + 1))

  return output

In [0]:
# class Embedder(object):
  
#   def __init__(self, BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=True):
#     self.BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#     self.tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)
#     self.trainable = trainable
    
#   def __call__(self, input_ids, input_mask, segment_ids):
#     embedder = hub.Module(self.BERT_model_hub,trainable=self.trainable)
#     bert_inputs = dict(input_ids=input_ids,
#                        input_mask=input_mask, 
#                        segment_ids=segment_ids)

#     bert_outputs = embedder(inputs=bert_inputs,
#                                signature="tokens", 
#                                as_dict=True)
    
#   # Use "pooled_output" for classification tasks on an entire sentence. Use "sequence_outputs" for token-level output
#     return bert_outputs["pooled_output"]
  

In [0]:
class Decoder(object):
  """The Decoder."""

  def __init__(self, layer_dims, num_classes):
    self.layer_dims = layer_dims
    self.num_classes = num_classes
    
  def __call__(self, input_xs_embedding, z_samples):
  
        # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]
    n_draws = z_samples.get_shape().as_list()[0]
    n_xs = tf.shape(input_xs_embedding)[0]

    # Repeat z samples for each x*
    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)

    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples, [1, n_xs, 1])

    # Repeat x* for each z sample
    x_star_repeat = tf.expand_dims(input_xs_embedding, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])

    # Concatenate x* and z
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)

    # decoder mlp
    inner_layer_dims = self.layer_dims
    output_dim = self.num_classes *2
    hidden = batch_mlp(inputs, inner_layer_dims, output_dim, "decoder")

    # Get the mean an the variance
    mu, log_sigma = tf.split(hidden, 2, axis= -1)

    # Bound the variance
    sigma_star = 0.1 + 0.9 * tf.nn.softplus(log_sigma)
    mu_star = tf.math.sigmoid(mu)


    return GaussianParams(mu_star, sigma_star)




In [0]:
class Encoder(object):

  def __init__(self, layer_dims, latent_dim):
    self.layer_dims = layer_dims
    self.latent_dim = latent_dim
    
  def __call__(self, xs, ys):
    print(xs)
    print(ys)
    xys = tf.concat([xs, ys], axis=1)


    # encoder mlp
    inner_layer_dims = self.layer_dims[:-1]
    output_dim = self.layer_dims[-1]
    rs = batch_mlp(xys, inner_layer_dims, output_dim, "encoder")
    
    # aggregate rs
    r = self._aggregate_r(rs)
    
    # get mu and sigma
    z_params = self._get_z_params(r)
    
    # distribution
    dist = tfd.MultivariateNormalDiag(loc=z_params.mu,
                                          scale_diag=z_params.sigma)
    return dist
    
  def _aggregate_r(self, context_rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation

    Creates an aggregation (mean) operator to combine the encodings of multiple context inputs

    Parameters
    ----------
    context_rs
        Input encodings tensor, shape: (n_samples, dim_r)

    Returns
    -------
        Output tensor of aggregation result
    """
    mean = tf.reduce_mean(context_rs, axis=0)
    r = tf.reshape(mean, [1, -1])
    return r
  
  def _get_z_params(self, context_r: tf.Tensor) -> GaussianParams:
    """Map encoding to mean and covariance of the random variable Z

    Creates a linear dense layer to map encoding to mu_z, and another linear mapping + a softplus activation for Sigma_z

    Parameters
    ----------
    context_r
        Input encoding tensor, shape: (1, dim_r)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
    hidden = context_r
    with tf.variable_scope("latent_encoder", reuse=tf.AUTO_REUSE):
      # First apply intermediate relu layer 
      hidden = tf.nn.relu(
          tf.layers.dense(hidden, 
                          (self.layer_dims[-1] + self.latent_dim)/2, 
                          name="penultimate_layer"))
      
      # Then apply further linear layers to output latent mu and log sigma
      mu = tf.layers.dense(hidden, self.latent_dim, name="mean_layer")
      log_sigma = tf.layers.dense(hidden, self.latent_dim, name="std_layer")
      

    # Compute sigma
    sigma = 0.1 + 0.9 * tf.sigmoid(log_sigma)

    return GaussianParams(mu, sigma)

In [0]:
class NLP_NeuralProcess(object):
  
  def __init__(self,
                  score_col, 
                  params = NeuralProcessParams(dim_z=20, 
                                                     n_hidden_units_h=[128, 128, 128], 
                                                     n_hidden_units_g=[128, 128, 128]),
                  num_classes = 6, 
                  num_draws = 2, 
                  lr = 2e-5,
                  batch_size = 32, 
                  num_warmup_steps=100,
                  num_train_steps = 10**3,
                  save_summary_steps = 100,
                  save_checkpoints_steps = 500,
                  output_dir = "./test_output",
                  context_features = None
                 ):
    
    self.params = params
    self.encoder = Encoder(layer_dims = self.params.n_hidden_units_h, 
                           latent_dim=self.params.dim_z)
    self.decoder = Decoder(layer_dims= self.params.n_hidden_units_g, 
                           num_classes=num_classes)
    self.num_draws = num_draws
    self.num_classes = num_classes
#     self.estimator = None
    #self.embedder = Embedder()
    #####  
    num_labels = len(score_col)
    
    
    # Specify outpit directory and number of checkpoint steps to save
    
    run_config = tf.estimator.RunConfig(model_dir=output_dir,
          save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)
    
    model_fn = self.model_fn_builder(num_labels = num_labels, learning_rate=lr,
      num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)
    
    if context_features is not None:
      estimator_params = {"batch_size": batch_size, "context_features": context_features}
      self.estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
                                              params=estimator_params)
    
    else: 
      self.estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
            params={"batch_size": batch_size})
    
    
  def create_model(self, 
                   target_input_ids, 
                   target_input_mask, 
                   target_segment_ids, 
                   target_scores=None,
                   context_input_ids = None, 
                   context_input_mask = None, 
                   context_segment_ids= None, 
                   context_scores = None
                   ):
    
    # apply embedder
    embedder = hub.Module(BERT_model_hub,trainable=True)
    
    valid_context = (context_input_ids is not None) & (context_input_mask is not None) & (context_segment_ids is not None) & (context_scores is not None)
    
    # target processing - all scenarios
    target_inputs = dict(input_ids=target_input_ids,
                     input_mask=target_input_mask, 
                     segment_ids=target_segment_ids)
    target_embeddings = embedder(inputs=target_inputs,
                               signature="tokens", 
                               as_dict=True)
    target_xs = target_embeddings["pooled_output"]
    
    
    if valid_context:
      
      # context processing - training
      context_inputs = dict(input_ids=context_input_ids,
                         input_mask=context_input_mask, 
                         segment_ids=context_segment_ids)
      context_embeddings = embedder(inputs=context_inputs,
                                 signature="tokens", 
                                 as_dict=True)
      context_xs = context_embeddings["pooled_output"]
      context_ys = context_scores
      # total x,y 
      x_all = tf.concat([context_xs, target_xs], axis=0)
      
      # get encoding params with context
      context_z_dist = self.encoder(context_xs, context_ys)
      # predictions with context
      posterior_pred = self.decoder(target_xs, context_z_dist.sample(self.num_draws))
        
       # target scores - context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        y_all = tf.concat([context_ys, target_ys], axis=0)
        all_z_dist = self.encoder(x_all, y_all)
        
        # loss
        loglike = self.loglikelihood(target_ys, posterior_pred)
        KL_loss = self.KLqp_gaussian(all_z_dist.parameters['loc'], 
                                     all_z_dist.parameters['scale_diag'], 
                                     context_z_dist.parameters['loc'], 
                                     context_z_dist.parameters['scale_diag'])
        loss = tf.negative(loglike) + KL_loss
        # context and training / evaluation
        return (loss, posterior_pred, target_ys)
      
      # context prediction
      return  (None, posterior_pred, None)
    
    # no context
    else:
      x_all = target_xs 
      # get internal representation
      mean_zero = tf.constant(np.repeat(0., params.dim_z))
      epsilon_dist = tfd.MultivariateNormalDiag(loc= mean_zero)                            
      epsilon = tf.expand_dims(epsilon_dist.sample(self.num_draws), axis=1)
      epsilon = tf.cast(epsilon, tf.float32)
      prior_predict = self.decoder(x_all, epsilon)
      
      # target scores - no context training / evaluation
      if target_scores is not None:
        target_ys = target_scores
        loglike = self.loglikelihood(target_ys, prior_predict)
        loss = tf.negative(loglike)
        # no context/ training / evaluation
        return  (loss, prior_predict, target_ys)
   
    
      # no context prediction
      return  (None, prior_predict, None)

  
  def loglikelihood(self, y_star: tf.Tensor, dist):
    """Log-likelihood of an output given a predicted """
    p_normal = tfd.MultivariateNormalDiag(loc = dist.mu, scale_diag=dist.sigma)
    loglike = p_normal.log_prob(y_star)
    loglike = tf.reduce_sum(loglike, axis=0)
    loglike = tf.reduce_mean(loglike)
    return loglike
  
  def KLqp_gaussian(self, mu_q: tf.Tensor, sigma_q: tf.Tensor, mu_p: tf.Tensor, sigma_p: tf.Tensor) -> tf.Tensor:
    """Kullback-Leibler divergence between two Gaussian distributions

    Determines KL(q || p) = < log( q / p ) >_q

    Parameters
    ----------
    mu_q
        Mean tensor of distribution q, shape: (1, dim)
    sigma_q
        Variance tensor of distribution q, shape: (1, dim)
    mu_p
        Mean tensor of distribution p, shape: (1, dim)
    sigma_p
        Variance tensor of distribution p, shape: (1, dim)

    Returns
    -------
        KL tensor, shape: (1)
    """
    sigma2_q = tf.square(sigma_q) + 1e-16
    sigma2_p = tf.square(sigma_p) + 1e-16
    temp = sigma2_q / sigma2_p + tf.square(mu_q - mu_p) / sigma2_p - 1.0 + tf.log(sigma2_p / sigma2_q + 1e-16)
    return 0.5 * tf.reduce_sum(temp)
  
  def context_target_split(self, batch_size =32):
    btch_sz = batch_size
    n_context = tf.random_shuffle(tf.range(1,btch_sz))[0]
    
    indices = tf.range(0, btch_sz)
    context_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context))
    target_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context, btch_sz))
    
    return context_set_indices, target_set_indices
    
  def model_fn_builder(self, num_labels, learning_rate, num_train_steps, num_warmup_steps):
      """Returns `model_fn` closure for TPUEstimator."""
      
      
      def model_fn(features, mode, params):  # pylint: disable=unused-argument
          """The `model_fn` for TPUEstimator."""
          
          # run model
          # -------------------------------------------------------------------------------------------
          target_input_ids = None
          target_input_mask = None
          target_segment_ids = None 
          target_scores = None
          
          context_input_ids = None 
          context_input_mask = None 
          context_segment_ids = None
          context_scores = None
          
          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
            input_ids = features["input_ids"]
            input_mask = features["input_mask"]
            segment_ids = features["segment_ids"]    
            scores = features["scores"]
            
            # context split
            context_set_indices, target_set_indices= self.context_target_split(batch_size =32)
            
            target_input_ids = tf.gather(input_ids,target_set_indices)
            target_input_mask = tf.gather(input_mask,target_set_indices)
            target_segment_ids = tf.gather(segment_ids,target_set_indices) 
            target_scores = tf.gather(scores,target_set_indices)

            context_input_ids = tf.gather(input_ids,context_set_indices) 
            context_input_mask = tf.gather(input_mask,context_set_indices) 
            context_segment_ids = tf.gather(segment_ids,context_set_indices)
            context_scores = tf.gather(scores,context_set_indices)
            
            
          elif mode == tf.estimator.ModeKeys.PREDICT:
            print('Prediction')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]
            
            try:
              context_input_ids = features["supplied_context_input_ids"]
              context_input_mask = features["supplied_context_input_mask"]
              context_segment_ids = features["supplied_context_segment_ids"] 
              context_scores = features["supplied_context_scores"]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None
            
          else:
            print('Evaluation')
            target_input_ids = features["input_ids"]
            target_input_mask = features["input_mask"]
            target_segment_ids = features["segment_ids"]    
            target_scores = features["scores"]

            try:
              context_input_ids = features["supplied_context_input_ids"]
              context_input_mask = features["supplied_context_input_mask"]
              context_segment_ids = features["supplied_context_segment_ids"] 
              context_scores = features["supplied_context_scores"]
               
            except:
              print("****No context supplied ****")
              context_input_ids = None
              context_input_mask = None
              context_segment_ids = None
              context_scores = None


          (loss, prediction, true_y) = self.create_model(target_input_ids, 
                                                         target_input_mask, 
                                                         target_segment_ids, 
                                                         target_scores,
                                                         context_input_ids, 
                                                         context_input_mask, 
                                                         context_segment_ids, 
                                                         context_scores)

          train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
          ystar, _ = tf.nn.moments(prediction.mu,[0])
          variance, _ = tf.nn.moments(tf.math.square(prediction.sigma),[0])
          
          # Calculate evaluation metrics
          eval_metrics = {}
          # AUC
          def metric_fn(pred_scores, real_scores, trait_num):
              auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
              accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
              return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value}

          labels = true_y # need to round them if true labels are not 1 or 0
          eval_metrics_lst = [metric_fn(ystar, labels, trait_num) for trait_num in range(num_labels)]

          for d in eval_metrics_lst:
              tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1])  # make available to tensorboard
              tf.summary.scalar(list(d.keys())[1], list(d.values())[1][1])  # make available to tensorboard
              eval_metrics.update(d)
              
          
          # output from model
          # -------------------------------------------------------------------------------------------

          # training 
          if mode == tf.estimator.ModeKeys.TRAIN:
              return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

          # prediction
          elif mode == tf.estimator.ModeKeys.PREDICT:
              return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': ystar, 'prediction_var': variance})

          # evaluation
          else:
            return tf.estimator.EstimatorSpec(mode=mode,loss=loss, eval_metric_ops=eval_metrics)

      return model_fn
  
#   def model_fn_builder(self, num_labels, learning_rate, num_train_steps, num_warmup_steps):
#       """Returns `model_fn` closure for TPUEstimator."""
      
      
#       def model_fn(features, mode, params):  # pylint: disable=unused-argument
#           """The `model_fn` for TPUEstimator."""
          
#           # run model
#           # -------------------------------------------------------------------------------------------
#           target_input_ids = None
#           target_input_mask = None
#           target_segment_ids = None 
#           target_scores = None
          
#           context_input_ids = None 
#           context_input_mask = None 
#           context_segment_ids = None
#           context_scores = None
          
#           # training 
#           if mode == tf.estimator.ModeKeys.TRAIN:
#             input_ids = features["input_ids"]
#             input_mask = features["input_mask"]
#             segment_ids = features["segment_ids"]    
#             scores = features["scores"]
            
#             # context split
#             context_set_indices, target_set_indices= self.context_target_split(batch_size =32)
            
#             target_input_ids = tf.gather(input_ids,target_set_indices)
#             target_input_mask = tf.gather(input_mask,target_set_indices)
#             target_segment_ids = tf.gather(segment_ids,target_set_indices) 
#             target_scores = tf.gather(scores,target_set_indices)

#             context_input_ids = tf.gather(input_ids,context_set_indices) 
#             context_input_mask = tf.gather(input_mask,context_set_indices) 
#             context_segment_ids = tf.gather(segment_ids,context_set_indices)
#             context_scores = tf.gather(scores,context_set_indices)
            
            
#           elif mode == tf.estimator.ModeKeys.PREDICT:
#             print('Prediction')
#             target_input_ids = features["input_ids"]
#             target_input_mask = features["input_mask"]
#             target_segment_ids = features["segment_ids"]    
#             target_scores = features["scores"]
            
#             try:
#               context_input_ids = []
#               context_input_mask = []
#               context_segment_ids = []
#               context_scores = []  
#               num_context = len(params["context_features"])
              
#               for feature in params["context_features"]:
#                 context_input_ids.append(feature.input_ids)
#                 context_input_mask.append(feature.input_mask)
#                 context_segment_ids.append(feature.segment_ids)
#                 context_scores.append(feature.score)
              
#               context_input_ids = tf.constant(context_input_ids, shape=[num_context, 128],
# 						dtype=tf.int32)
              
#               context_input_mask = tf.constant(context_input_mask, shape=[num_context, 128],
# 						dtype=tf.int32)
#               context_segment_ids = tf.constant(context_segment_ids, shape=[num_context, 128],
# 						dtype=tf.int32)
#               context_scores = tf.constant(context_scores, shape=[num_context, self.num_classes])
              
#             except:
#               print("****No context supplied ****")
#               context_input_ids = None
#               context_input_mask = None
#               context_segment_ids = None
#               context_scores = None
            
#           else:
#             print('Evaluation')
#             target_input_ids = features["input_ids"]
#             target_input_mask = features["input_mask"]
#             target_segment_ids = features["segment_ids"]    
#             target_scores = features["scores"]

#             try:
#               context_input_ids = features["supplied_context_input_ids"]
#               context_input_mask = features["supplied_context_input_mask"]
#               context_segment_ids = features["supplied_context_segment_ids"] 
#               context_scores = features["supplied_context_scores"]
               
#             except:
#               print("****No context supplied ****")
#               context_input_ids = None
#               context_input_mask = None
#               context_segment_ids = None
#               context_scores = None


#           (loss, prediction, true_y) = self.create_model(target_input_ids, 
#                                                          target_input_mask, 
#                                                          target_segment_ids, 
#                                                          target_scores,
#                                                          context_input_ids, 
#                                                          context_input_mask, 
#                                                          context_segment_ids, 
#                                                          context_scores)

#           train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
#           ystar, _ = tf.nn.moments(prediction.mu,[0])
#           variance, _ = tf.nn.moments(tf.math.square(prediction.sigma),[0])
          
          
#           # output from model
#           # -------------------------------------------------------------------------------------------

#           # training 
#           if mode == tf.estimator.ModeKeys.TRAIN:
#               return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

#           # prediction
#           elif mode == tf.estimator.ModeKeys.PREDICT:
#               return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': ystar, 'prediction_var': variance})

#           # evaluation
#           else:
#             # Calculate evaluation metrics.
#             eval_metrics = {}

#             # AUC
#             def metric_fn(pred_scores, real_scores, trait_num):
#                 auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
#                 accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
#                 return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value}

#             labels = true_y # need to round them if true labels are not 1 or 0
#             eval_metrics_lst = [metric_fn(ystar, labels, trait_num) for trait_num in range(num_labels)]

#             for d in eval_metrics_lst:
#                 tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1])  # make available to tensorboard
#                 tf.summary.scalar(list(d.keys())[1], list(d.values())[1][1])  # make available to tensorboard
#                 eval_metrics.update(d)
#             return tf.estimator.EstimatorSpec(mode=mode,loss=loss, eval_metric_ops=eval_metrics)

#       return model_fn

  def prepare_examples(self, df, score_column, text_col_name, supplied_context_df=None):
      num_labels = len(score_column)
      input_examples = create_examples(df, score_column, text_col_name)
      input_features = convert_examples_to_features(input_examples, max_seq_length, tokenizer)
      
      if supplied_context_df is not None:
        supplied_context_examples = create_examples(supplied_context_df, score_column, text_col_name)
        supplied_context_features = convert_examples_to_features(supplied_context_examples, max_seq_length, tokenizer)
        input_fn = input_fn_builder(
          features=input_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False,
          supplied_context_features = supplied_context_features)
      
      else:
        input_fn = input_fn_builder(
          features=input_features, seq_length=max_seq_length, 
          num_labels = num_labels, is_training=True, drop_remainder=False)
        
      return input_fn
  
  def predict(self, 
            df, 
            score_col, 
            text_col,
            supplied_context_df=None
             ):
    
    pred_input_fn = self.prepare_examples(df, score_col, text_col, supplied_context_df)
    preds = self.estimator.predict(input_fn=pred_input_fn)
    
    return preds
  
  def evaluate(self, 
               eval_steps,
               df, 
               score_col, 
               text_col,
               supplied_context_df=None):
    
    eval_input_fn = self.prepare_examples(df, score_col, text_col, supplied_context_df)
    
    result = self.estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
    return result
  
  def train(self,num_train_steps,
            df_train, 
            score_col, 
            text_col,
           ):

    # Create an input function for training. drop_remainder = True for using TPUs.
    train_input_fn = self.prepare_examples(df_train, score_col, text_col)

    print('Beginning Training!')
    current_time = datetime.now()
    self.estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
    print("Training took time ", datetime.now() - current_time)

In [17]:

# tf.reset_default_graph()

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 
text_col_name = 'comment_text'

params = NeuralProcessParams(dim_z=1000, n_hidden_units_h=[512, 256, 128], n_hidden_units_g=[512, 256, 128])
num_train_steps = 12*(10**3)

neural_process = NLP_NeuralProcess(score_col=score_column, params = params, num_draws = 20, 
                                   num_train_steps=num_train_steps,
                                   output_dir = "./gdrive/My Drive/Kaggle_toxic_comments/test_output",
                                  context_features = None)

### Train:

neural_process.train(df_train=df_train, 
                     score_col= score_column, 
                     text_col=text_col_name, 
                     num_train_steps=num_train_steps) 

INFO:tensorflow:Using config: {'_model_dir': './gdrive/My Drive/Kaggle_toxic_comments/test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f04d511cb38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0528 09:25:49.777951 139661544327040 estimator.py:201] Using config: {'_model_dir': './gdrive/My Drive/Kaggle_toxic_comments/test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f04d511cb38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Beginning Training!
INFO:tensorflow:Calling model_fn.


I0528 09:26:54.965257 139661544327040 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 12000 into ./gdrive/My Drive/Kaggle_toxic_comments/test_output/model.ckpt.


I0528 12:09:55.922056 139661544327040 basic_session_run_hooks.py:594] Saving checkpoints for 12000 into ./gdrive/My Drive/Kaggle_toxic_comments/test_output/model.ckpt.


INFO:tensorflow:Loss for final step: -166.0372.


I0528 12:10:08.234185 139661544327040 estimator.py:359] Loss for final step: -166.0372.


Training took time  2:43:31.629822


In [18]:
df_test = test_df#.sample(120)

# supp_context = df_train.sample(100)

batch_size = 32
eval_steps = int(len(df_test) / batch_size)

neural_process.evaluate(eval_steps,
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name, supplied_context_df = None)

INFO:tensorflow:Calling model_fn.


I0528 12:24:50.317733 139661544327040 estimator.py:1111] Calling model_fn.


Evaluation
****No context supplied ****
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 12:24:53.567503 139661544327040 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 12000: accuracy0 = 0.89262336, accuracy1 = 0.98495173, accuracy2 = 0.94662017, accuracy3 = 0.99321353, accuracy4 = 0.9415236, accuracy5 = 0.983074, auc0 = 0.96047705, auc1 = 0.98244643, auc2 = 0.97237784, auc3 = 0.8966579, auc4 = 0.9696056, auc5 = 0.8407147, global_step = 12000, loss = -103.14413


I0528 12:31:08.124746 139661544327040 estimator.py:1979] Saving dict for global step 12000: accuracy0 = 0.89262336, accuracy1 = 0.98495173, accuracy2 = 0.94662017, accuracy3 = 0.99321353, accuracy4 = 0.9415236, accuracy5 = 0.983074, auc0 = 0.96047705, auc1 = 0.98244643, auc2 = 0.97237784, auc3 = 0.8966579, auc4 = 0.9696056, auc5 = 0.8407147, global_step = 12000, loss = -103.14413


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12000: ./gdrive/My Drive/Kaggle_toxic_comments/test_output/model.ckpt-12000


I0528 12:31:11.666625 139661544327040 estimator.py:2039] Saving 'checkpoint_path' summary for global step 12000: ./gdrive/My Drive/Kaggle_toxic_comments/test_output/model.ckpt-12000


{'accuracy0': 0.89262336,
 'accuracy1': 0.98495173,
 'accuracy2': 0.94662017,
 'accuracy3': 0.99321353,
 'accuracy4': 0.9415236,
 'accuracy5': 0.983074,
 'auc0': 0.96047705,
 'auc1': 0.98244643,
 'auc2': 0.97237784,
 'auc3': 0.8966579,
 'auc4': 0.9696056,
 'auc5': 0.8407147,
 'global_step': 12000,
 'loss': -103.14413}

In [0]:
# ! cp -r ./test_output "./gdrive/My Drive/Kaggle_toxic_comments/"

In [0]:
preds = neural_process.predict(
                       df_test, 
                       score_col= score_column, 
                       text_col=text_col_name,supplied_context_df = None)

In [21]:
i=0
for pred in preds:
  i += 1
  if i<10:
    print(pred)
  else:
    break

INFO:tensorflow:Calling model_fn.


I0528 13:08:29.141242 139661544327040 estimator.py:1111] Calling model_fn.


Prediction
****No context supplied ****
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 13:08:32.729330 139661544327040 saver.py:1483] Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


I0528 13:08:47.632882 139661544327040 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0528 13:08:47.882883 139661544327040 session_manager.py:493] Done running local_init_op.


{'prediction_mean': array([9.8771753e-04, 8.2227739e-04, 7.6934844e-03, 2.7445262e-02,
       2.8924644e-05, 9.2196189e-02], dtype=float32), 'prediction_var': array([0.01002009, 0.0100431 , 0.0100469 , 0.01001257, 0.01000196,
       0.01001668], dtype=float32)}
{'prediction_mean': array([6.34933996e-04, 8.83944333e-04, 8.66532512e-03, 4.09781672e-02,
       2.21207738e-05, 1.12177834e-01], dtype=float32), 'prediction_var': array([0.01000948, 0.01005954, 0.01004871, 0.01001398, 0.01000148,
       0.010012  ], dtype=float32)}
{'prediction_mean': array([0.92653036, 0.03144516, 0.8688755 , 0.0486189 , 0.7748625 ,
       0.10828374], dtype=float32), 'prediction_var': array([0.0171299 , 0.03541978, 0.432202  , 0.01083835, 0.23058406,
       0.0558685 ], dtype=float32)}
{'prediction_mean': array([5.2458793e-04, 8.9413521e-04, 8.7813903e-03, 4.9624957e-02,
       2.1216274e-05, 1.2607256e-01], dtype=float32), 'prediction_var': array([0.01000761, 0.010068  , 0.01005081, 0.010015  , 0.01000133,


In [0]:
test_scores = df_test[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']]

In [0]:
uncertainty = pd.DataFrame(columns=['accuracy', 'variance'])
pred_means = pd.DataFrame(columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'])
pred_variances = pd.DataFrame(columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'])

for i, pred in enumerate(preds):
  if i<len(test_scores):
    scores = np.array(test_scores.iloc[i])
    #     print(scores)
    mean_prediction = np.round(pred['prediction_mean'])
    pred_means = pred_means.append({score_column[k]:pred['prediction_mean'][k] for k in range(len(score_column))}, ignore_index=True)
    #     print(mean_prediction)
    acc = scores==mean_prediction
    variance = pred['prediction_var']
    pred_variances = pred_variances.append({score_column[k]:variance[k] for k in range(len(score_column))}, ignore_index=True)
    for j in range(len(scores)):
      uncertainty = uncertainty.append({'accuracy': acc[j], 'variance': variance[j]}, ignore_index=True)
  else:
    break

In [106]:
print(uncertainty[uncertainty.accuracy==True]['variance'].mean())
print(uncertainty[uncertainty.accuracy==False]['variance'].mean())

0.03968876489706338
0.2590321116743508


In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  comparison = preds == test_scores
  correct_guesses = np.reshape(comparison.values,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  comparison = preds == test_scores
  correct_guesses = comparison.values
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [109]:
print(total_accuaracy(pred_means, test_scores.reset_index(drop=True)))
print(class_accuaracy(pred_means, test_scores.reset_index(drop=True)))

0.8877807269185054
[0.71103783 0.97731843 0.83179174 0.98662163 0.83903054 0.98088421]


In [133]:
def subset_accuracy(att):
  print("accuracy on "+att+ " comments:")
  print(class_accuaracy(pred_means.iloc[np.where(test_scores[att] == 1)].reset_index(drop=True),
                        test_scores.iloc[np.where(test_scores[att] == 1)].reset_index(drop=True)))

  print("proportion of "+att+ " comments:")
  print(len(test_scores[test_scores[att] ==1])/len(test_scores))
  print()

for att in score_column:
  subset_accuracy(att)

accuracy on toxic comments:
[0.22029045 0.93495602 0.42667212 0.95868276 0.46430763 0.88668439]
proportion of toxic comments:
0.13107590015818119

accuracy on severe_toxic comments:
[0.19650655 0.         0.09606987 0.89519651 0.18340611 0.65938865]
proportion of severe_toxic comments:
0.006139574787527816

accuracy on obscene comments:
[0.22695035 0.9047619  0.11043566 0.95812226 0.31273219 0.86018237]
proportion of obscene comments:
0.0793855063138422

accuracy on threat comments:
[0.23353293 0.83233533 0.40718563 0.01796407 0.38922156 0.79041916]
proportion of threat comments:
0.004477331831952599

accuracy on insult comments:
[0.23107715 0.90829694 0.27620087 0.95487627 0.10953421 0.82860262]
proportion of insult comments:
0.07367489745033379

accuracy on identity_hate comments:
[0.22504537 0.83666062 0.33212341 0.9292196  0.21778584 0.00181488]
proportion of identity_hate comments:
0.014772514008418456



In [137]:
len(test_scores[test_scores.severe_toxic==1])

229

In [0]:
at_least_one = (test_scores.toxic==1) | (test_scores.severe_toxic==1) | \
               (test_scores.obscene==1) | (test_scores.threat==1) | \
               (test_scores.insult==1) | (test_scores.identity_hate==1)

In [0]:
print(uncertainty[uncertainty.accuracy==True]['variance'].mean())
print(uncertainty[uncertainty.accuracy==False]['variance'].mean())

37299

In [0]:
try: 
  print("this")
  True/"this"
except:
  print("didnt work")

this
didnt work
